# Machine Learning Experimentation with MLflow, TensorFlow, and CatBoost

In this tutorial, we demonstrate a comprehensive machine learning workflow using MLflow for experiment tracking, TensorFlow for building neural network models, and CatBoost for gradient boosting models. Our objective is to predict the median income of households in California districts, based on several features such as median income, housing average age, and geographical location.

## Experiment Setup with MLflow

MLflow is an open-source platform for managing the end-to-end machine learning lifecycle. It includes features for experiment tracking, model versioning, and deployment. MLflow helps in comparing different models and tracking experiments to ensure reproducibility.

```python
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("income")


In [12]:
import mlflow
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import tensorflow_addons as tfa

from tensorflow.keras.callbacks import EarlyStopping

from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset

import catboost as cb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError

In [13]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 15})

In [14]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("income")

<Experiment: artifact_location='file:///d:/Mlflow/tutorials-main/mlflow/tutorials/mlflow/mlruns/1', creation_time=1707252810494, experiment_id='1', last_update_time=1707252810494, lifecycle_stage='active', name='income', tags={}>

## Load Data

In [15]:
dset = fetch_california_housing()
data = dset['data']
y = dset['target']
LABEL = dset['target_names'][0]

NUMERIC_FEATURES = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
FEATURES = NUMERIC_FEATURES

data = pd.DataFrame(data, columns=dset['feature_names'])
data[LABEL] = y

data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [16]:
train_data, test_data = train_test_split(data, test_size=0.2)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (16512, 9)
Test dataset shape: (4128, 9)


## Data Processing

## Feature Scaling

Before training our models, it's crucial to apply feature scaling to ensure all numeric features contribute equally to the model's performance. This is especially important for models that are sensitive to the scale of the input features, such as neural networks and models that use distance measures.




In [17]:
X_train, X_val = train_test_split(train_data, test_size=0.2)

sc = StandardScaler()
X_train.loc[:, NUMERIC_FEATURES] = sc.fit_transform(X_train[NUMERIC_FEATURES])
X_val.loc[:, NUMERIC_FEATURES] = sc.transform(X_val[NUMERIC_FEATURES])
test_data.loc[:, NUMERIC_FEATURES] = sc.transform(test_data[NUMERIC_FEATURES])

## Baseline
### Model with RandomForestRegressor
The RandomForestRegressor serves as our baseline model. It's an ensemble method that builds multiple decision trees and merges them for a more accurate and robust prediction. This model is known for its ability to handle complex datasets with a mix of numerical and categorical features effectively.





In [18]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)

    rf = RandomForestRegressor(n_estimators=100, max_depth=20)
    rf.fit(X_train[FEATURES], X_train[LABEL])

    rf_preds = rf.predict(test_data[FEATURES])
    rf_rms = mean_squared_error(test_data[LABEL], rf_preds, squared=False)

    mlflow.log_metric("test_rmse", rf_rms)
    mlflow.sklearn.log_model(rf, "sk_models")

c:\Users\User\anaconda3\envs\mlflow_env\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### CatBoost Model
Next, we use CatBoost, a gradient boosting library that excels in dealing with categorical features directly, without the need for extensive preprocessing. CatBoost is designed to provide state-of-the-art results with minimal tuning and is particularly user-friendly for both regression and classification tasks.


In [19]:
catb_train_dataset = cb.Pool(X_train[FEATURES], X_train[LABEL]) 
catb_val_dataset = cb.Pool(X_val[FEATURES], X_val[LABEL]) 
catb_test_dataset = cb.Pool(test_data[FEATURES], test_data[LABEL])

In [20]:
with mlflow.start_run(run_name="catboost"):
    mlflow.set_tag("model_name", "CatBoost")
    catb = cb.CatBoostRegressor()
    catb.fit(catb_train_dataset, eval_set=catb_val_dataset, early_stopping_rounds=50)
    catb_preds = catb.predict(catb_test_dataset)
    catb_rms = mean_squared_error(test_data[LABEL], catb_preds, squared=False)

    mlflow.log_metric("test_rmse", catb_rms)
    mlflow.catboost.log_model(catb, "cb_models")

Learning rate set to 0.076361
0:	learn: 1.1096733	test: 1.1081870	best: 1.1081870 (0)	total: 153ms	remaining: 2m 32s
1:	learn: 1.0663889	test: 1.0655551	best: 1.0655551 (1)	total: 157ms	remaining: 1m 18s
2:	learn: 1.0247556	test: 1.0247016	best: 1.0247016 (2)	total: 160ms	remaining: 53.3s
3:	learn: 0.9873031	test: 0.9885692	best: 0.9885692 (3)	total: 163ms	remaining: 40.6s
4:	learn: 0.9518877	test: 0.9538915	best: 0.9538915 (4)	total: 166ms	remaining: 33.1s
5:	learn: 0.9207428	test: 0.9233539	best: 0.9233539 (5)	total: 169ms	remaining: 28s
6:	learn: 0.8934503	test: 0.8969540	best: 0.8969540 (6)	total: 172ms	remaining: 24.4s
7:	learn: 0.8673118	test: 0.8711149	best: 0.8711149 (7)	total: 175ms	remaining: 21.7s
8:	learn: 0.8438365	test: 0.8478673	best: 0.8478673 (8)	total: 178ms	remaining: 19.6s
9:	learn: 0.8232280	test: 0.8277919	best: 0.8277919 (9)	total: 181ms	remaining: 17.9s
10:	learn: 0.8040490	test: 0.8089706	best: 0.8089706 (10)	total: 185ms	remaining: 16.6s
11:	learn: 0.7840433	t

c:\Users\User\anaconda3\envs\mlflow_env\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Multi-layer Perceptron (MLP) Model
Finally, we explore a Multi-layer Perceptron (MLP), a type of neural network that consists of at least three layers of nodes: an input layer, hidden layers, and an output layer. MLPs can capture complex relationships in data by adjusting weights through backpropagation. In this context,

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping
import mlflow
from sklearn.metrics import mean_squared_error

def build_mlp(params):
    """
    Builds a Multi-layer Perceptron model based on provided parameters.
    
    :param params: A dictionary containing model configuration like layer sizes and dropout rate.
    :return: A compiled Keras Sequential model.
    """
    # Define the MLP model architecture
    mlp = Sequential([
        Dense(params["layer1_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer2_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(1, activation='relu')  # Output layer for regression
    ])
    return mlp

def train_mlp(mlp, train_params, train_dataset, val_dataset):
    """
    Compiles and trains the MLP model using provided training parameters and datasets.
    
    :param mlp: The MLP model to be trained.
    :param train_params: A dictionary with training configuration such as learning rate and weight decay.
    :param train_dataset: The dataset for training the model.
    :param val_dataset: The dataset for validating the model performance.
    :return: The trained MLP model.
    """
    # Configure the optimizer with weight decay
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )

    # Compile the model with loss and metrics
    mlp.compile(
        optimizer=optimizer,
        loss=MeanSquaredError(name="mse"),
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor="val_rmse",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )

    # Train the model
    mlp.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=[early_stopping],
    )
    return mlp

def mlp_mlflow_run(name, mlp_params, train_params, train_dataset, val_dataset, test_dataset, y_test):
    """
    Wrapper function to train the MLP model and log the experiment with MLflow.
    
    :param name: The name of the MLflow run.
    :param mlp_params: Model parameters for building the MLP.
    :param train_params: Training parameters.
    :param train_dataset: Training dataset.
    :param val_dataset: Validation dataset.
    :param test_dataset: Test dataset for final evaluation.
    :param y_test: True labels for the test dataset.
    """
    with mlflow.start_run(run_name=name):
        mlflow.log_params(mlp_params)
        mlflow.log_params(train_params)
        mlflow.set_tag("model_name", "MLP")

        # Build, train, and evaluate the model
        mlp = build_mlp(mlp_params)
        mlp = train_mlp(mlp, train_params, train_dataset, val_dataset)

        test_preds = mlp.predict(test_dataset)
        test_rms = mean_squared_error(y_test, test_preds.ravel(), squared=False)
        

        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(mlp, "tf_models")

In [ ]:
# To TF Dataset
mlp_train_ds = tf.data.Dataset.from_tensor_slices((X_train[FEATURES], X_train[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_val_ds = tf.data.Dataset.from_tensor_slices((X_val[FEATURES], X_val[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_test_ds = tf.data.Dataset.from_tensor_slices(test_data[FEATURES]).batch(512).prefetch(512)

mlp_params = {
    "layer1_size": 512,
    "layer2_size": 128,
    "layer3_size": 64,
    "dropout_rate": 0.3,
    "activation": 'relu'

}
train_params = dict(
    learning_rate=0.008, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

mlp_mlflow_run(
    "mlp_base",
    mlp_params,
    train_params,
    mlp_train_ds,
    mlp_val_ds,
    mlp_test_ds,
    test_data[LABEL],
)

In [ ]:

mlp_params = {
    "layer1_size": 512,
    "layer2_size": 264,
    "layer3_size": 64,
    "dropout_rate": 0.1,
    "activation": 'relu'

}
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=30, num_epochs=1000
)

mlp_mlflow_run(
    "mlp_base_5",
    mlp_params,
    train_params,
    mlp_train_ds,
    mlp_val_ds,
    mlp_test_ds,
    test_data[LABEL],
)

In [ ]:
mlp_mlflow_run

## FT Transformers

In [54]:
# To TF Dataset
import tensorflow as tf
import numpy as np

def df_to_dataset(dataframe, target=None, shuffle=True, batch_size=32):
    df = dataframe.copy()
    if target:
        labels = df.pop(target).values
        # Convert dataframe to a dictionary of series, then to a dictionary of numpy arrays
        dataset = {key: value.values[:, np.newaxis] for key, value in df.items()}  # Adjusted line
        dataset = tf.data.Dataset.from_tensor_slices((dataset, labels))
    else:
        dataset = {key: value.values[:, np.newaxis] for key, value in df.items()}  # Adjusted line
        dataset = tf.data.Dataset.from_tensor_slices(dataset)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL, shuffle=True)
val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle

In [55]:
def build_fttransformer(
    params_to_log, params_to_skip, out_dim=1, out_activation="relu"
):
    """
    Builds an FTTransformer model with specified parameters.

    Parameters:
    - params_to_log: Dictionary of parameters that will be logged and used in the FTTransformerEncoder.
    - params_to_skip: Dictionary of parameters to skip during logging but used in the FTTransformerEncoder.
    - out_dim: Output dimension of the final layer. Default is 1.
    - out_activation: Activation function for the output layer. Default is "relu".

    Returns:
    - An FTTransformer model ready for training.
    """
    # Define encoder
    ft_encoder = FTTransformerEncoder(
        **params_to_log,
        **params_to_skip
    )
    # Add prediction head to the encoder
    ft_transformer = FTTransformer(
        encoder=ft_encoder,
        out_dim=out_dim,
        out_activation=out_activation,
    )

    return ft_transformer


def train_model(model, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )
    """
    Compiles and trains the given model using specified parameters and datasets.

    Parameters:
    - model: The model to train.
    - train_params: Training parameters including learning rate and weight decay.
    - train_dataset: The dataset for training the model.
    - val_dataset: The dataset for validating the model performance.

    Returns:
    - The trained model.
    """

    model.compile(
        optimizer=optimizer,
        loss={
            "output": tf.keras.losses.MeanSquaredError(name="mse"),
            "importances": None,
        },
        metrics={
            "output": [tf.keras.metrics.RootMeanSquaredError(name="rmse")],
            "importances": None,
        },
    )

    early = EarlyStopping(
        monitor="val_output_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = model.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return model


In [56]:
mlflow.tensorflow.autolog(disable=True)


def fttransformer_mlflow_run(
    name,
    encoder_params,
    train_params,
    params_to_skip,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", "FTTransformer")
        # Log the params
        mlflow.log_params(encoder_params)
        mlflow.log_params(train_params)
        # Build and train
        ft_transformer = build_fttransformer(
            encoder_params,
            params_to_skip,
            out_dim=1,
            out_activation="relu",
        )
        ft_transformer = train_model(
            ft_transformer, train_params, train_dataset, val_dataset
        )
        # Evaluate
        test_preds = ft_transformer.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds["output"].ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(ft_transformer, "tf_models")


In [57]:
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

params_to_skip = dict(
    numerical_data=X_train[NUMERIC_FEATURES].values,
    categorical_data=None,
    y=X_train[LABEL].values
)

### Linear Embeddings

In [58]:
linear_embeddings_params = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type="linear",
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='linear',
    encoder_params=linear_embeddings_params,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000



WARNI [tensorflow] From c:\Users\User\anaconda3\envs\mlflow_env\lib\site-packages\keras\src\layers\normalization\layer_normalization.py:328: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



WARNI [tensorflow] From c:\Users\User\anaconda3\envs\mlflow_env\lib\site-packages\keras\src\engine\base_layer_utils.py:384: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.



413/413 [==============================] - 14s 23ms/step - loss: 0.6985 - output_loss: 0.6985 - output_rmse: 0.8358 - val_loss: 0.5359 - val_output_loss: 0.5359 - val_output_rmse: 0.7321
Epoch 2/1000
413/413 [==============================] - 9s 22ms/step - loss: 0.4720 - output_loss: 0.4720 - output_rmse: 0.6870 - val_loss: 0.4025 - val_output_loss: 0.4025 - val_output_rmse: 0.6344
Epoch 3/1000
413/413 [==============================] - 9s 22ms/step - loss: 0.4503 - output_loss: 0.4503 - output_rmse: 0.6710 - val_loss: 0.3751 - val_output_loss: 0.3751 - val_output_rmse: 0.6125
Epoch 4/1000
413/413 [==============================] - 9s 22ms/step - loss: 0.4048 - output_loss: 0.4048 - output_rmse: 0.6363 - val_loss: 0.3820 - val_output_loss: 0.3820 - val_output_rmse: 0.6180
Epoch 5/1000
413/413 [==============================] - 10s 23ms/step - loss: 0.3953 - output_loss: 0.3953 - output_rmse: 0.6287 - val_loss: 0.3752 - val_output_loss: 0.3752 - val_output_rmse: 0.6125
Epoch 6/1000
413

c:\Users\User\anaconda3\envs\mlflow_env\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/02/06 22:04:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


...

WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


### Periodic

In [ ]:
periodic_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='periodic',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='periodic',
    encoder_params=periodic_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

### PLE - Quantile Binning

In [60]:
pleq_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

pleq_params_to_skip = params_to_skip.copy()
pleq_params_to_skip['y'] = None

fttransformer_mlflow_run(
    name='ple_quantile',
    encoder_params=pleq_params_to_log,
    train_params=train_params,
    params_to_skip=pleq_params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
413/413 [==============================] - 91s 106ms/step - loss: 0.4943 - output_loss: 0.4943 - output_rmse: 0.7030 - val_loss: 0.3435 - val_output_loss: 0.3435 - val_output_rmse: 0.5861
Epoch 2/1000
413/413 [==============================] - 22s 54ms/step - loss: 0.3317 - output_loss: 0.3317 - output_rmse: 0.5759 - val_loss: 0.3362 - val_output_loss: 0.3362 - val_output_rmse: 0.5799
Epoch 3/1000
413/413 [==============================] - 23s 56ms/step - loss: 0.2961 - output_loss: 0.2961 - output_rmse: 0.5442 - val_loss: 0.2785 - val_output_loss: 0.2785 - val_output_rmse: 0.5278
Epoch 4/1000
413/413 [==============================] - 23s 56ms/step - loss: 0.2752 - output_loss: 0.2752 - output_rmse: 0.5246 - val_loss: 0.3081 - val_output_loss: 0.3081 - val_output_rmse: 0.5551
Epoch 5/1000
413/413 [==============================] - 26s 63ms/step - loss: 0.2689 - output_loss: 0.2689 - output_rmse: 0.5185 - val_loss: 0.2801 - val_output_loss: 0.2801 - val_output_rmse: 0.5292

c:\Users\User\anaconda3\envs\mlflow_env\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/02/06 22:30:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


WARNI [tensorflow] Skipping full serialization of Keras layer <keras.src.layers.reshaping.flatten.Flatten object at 0x000001DBF76C8CD0>, because it is not built.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


...

WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


WARNI [tensorflow] Model's `__init__()` arguments contain non-serializable objects. Please implement a `get_config()` method in the subclassed Model for proper saving and loading. Defaulting to empty config.


### PLE - Target Binning

In [61]:
plet_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
    task='regression',
    ple_tree_params = {
        "min_samples_leaf": 20,
    }
)


fttransformer_mlflow_run(
    name='ple_target',
    encoder_params=plet_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

Epoch 1/1000
413/413 [==============================] - 104s 129ms/step - loss: 0.5015 - output_loss: 0.5015 - output_rmse: 0.7082 - val_loss: 0.3810 - val_output_loss: 0.3810 - val_output_rmse: 0.6173
Epoch 2/1000
413/413 [==============================] - 45s 108ms/step - loss: 0.3343 - output_loss: 0.3343 - output_rmse: 0.5782 - val_loss: 0.3008 - val_output_loss: 0.3008 - val_output_rmse: 0.5485
Epoch 3/1000
413/413 [==============================] - 46s 110ms/step - loss: 0.3002 - output_loss: 0.3002 - output_rmse: 0.5479 - val_loss: 0.2883 - val_output_loss: 0.2883 - val_output_rmse: 0.5369
Epoch 4/1000
413/413 [==============================] - 44s 107ms/step - loss: 0.2819 - output_loss: 0.2819 - output_rmse: 0.5310 - val_loss: 0.2718 - val_output_loss: 0.2718 - val_output_rmse: 0.5213
Epoch 5/1000
413/413 [==============================] - 44s 107ms/step - loss: 0.2649 - output_loss: 0.2649 - output_rmse: 0.5146 - val_loss: 0.2627 - val_output_loss: 0.2627 - val_output_rmse: 0

In [ ]:
model_id = "" # take it from mlflow
loaded_ft = mlflow.tensorflow.load_model(model_id)